In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import graphviz
import pydotplus
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import export_text
from sklearn.tree import export_graphviz
from IPython.display import Image, display

in the first cell all the libraries needed for the development of the code are imported

In [ ]:
df = pd.read_csv('/content/kaggle.csv')
df.head()

,Lat,Lng,What Dinosaurs Eat,Accepted Name,Country,Cc,Diet,Early Interval,Formation,Geological Interval,Geological Time Period,Ref Author,Ref Pubyr,State,Max Ma,Min Ma
0,42.933300,123.966698,PLANT,Chaoyangsaurus youngi,China,CN,herbivore,Late Tithonian,Tuchengzi,Tithonian,Jurassic,Dong,1992,Liaoning,150.8,132.90
1,41.799999,120.733330,PLANT and ANIMAL,Protarchaeopteryx robusta,China,CN,omnivore,Late Barremian,Yixian,Barremian,Cretaceous,Ji et al.,1998,Liaoning,130.0,122.46
2,41.799999,120.733330,PLANT and ANIMAL,Caudipteryx zoui,China,CN,omnivore,Late Barremian,Yixian,Barremian,Cretaceous,Ji and Ji,1997,Liaoning,130.0,122.46
3,50.740726,-111.528732,FLESH,Gorgosaurus libratus,Canada,CA,carnivore,Late Campanian,Dinosaur Park,Campanian,Cretaceous,Matthew and Brown,1922,Alberta,83.5,70.60
4,50.737015,-111.549347,FLESH,Gorgosaurus libratus,Canada,CA,carnivore,Late Campanian,Dinosaur Park,Campanian,Cretaceous,Russell,1970,Alberta,83.5,70.60


The I create a dataframe from pandas and all the information of the csv was saved it there


In [ ]:
deleted =["Ref Author","Ref Pubyr", "State","What Dinosaurs Eat", 'Diet']
x = df.drop(deleted, axis=1)
y = df['Diet']

print('Data divided')
print(x.columns)

Data divided
Index(['Lat', 'Lng', 'Accepted Name', 'Country', 'Cc', 'Early Interval',
       'Formation', 'Geological Interval', 'Geological Time Period', 'Max Ma',
       'Min Ma'],
      dtype='object')


The next step is to create features and save them in a variable x and my classes in a variable y; All the columns of the dataframe that are not used are deleted with a drop and saved in a variable called 'deleted' variable that will later be saved in x, as my intention is to classify dinosaurs by their type of diet, then the 'diet' column will be will become my classes column.

In [ ]:
label_encoder = LabelEncoder()
x['Accepted Name']=label_encoder.fit_transform(x['Accepted Name'])
x['Country']=label_encoder.fit_transform(x['Country'])
x['Cc']=label_encoder.fit_transform(x['Cc'])
x['Early Interval']=label_encoder.fit_transform(x['Early Interval'])
x['Formation']=label_encoder.fit_transform(x['Formation'])
x['Geological Interval']=label_encoder.fit_transform(x['Geological Interval'])
x['Geological Time Period']=label_encoder.fit_transform(x['Geological Time Period'])
x['Max Ma']=label_encoder.fit_transform(x['Max Ma'])
x['Min Ma']=label_encoder.fit_transform(x['Min Ma'])

imputer = SimpleImputer(strategy = "mean")

x_imputed = imputer.fit_transform(x)

scaler = StandardScaler()
x_scaler = scaler.fit_transform(x_imputed)

Having most of the information in Spanish, it is necessary to encode it so that the model can work with numerical values, so we use an imputer with the "mean" strategy to impute missing values in my data set 'x';
A Scaler is also used in which the characteristics of a data set are adjusted so that they have a mean of 0 and a standard deviation of 1.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaler, y, test_size=0.2, random_state=42)

As the next step, the division of my set 'x' and 'y' into train and test is carried out in order to train the model and then perform tests.

In [ ]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)


DecisionTreeClassifier()

The decision tree model is imported from the sklearn library and its training is carried out.

In [ ]:
y_pred = model.predict(x_test)

dinosaur_names = df['Accepted Name'].tolist()
for i in range(4):
    print(f'Dinosaurio: {dinosaur_names[i]}, Clasificación: {y_pred[i]}')


Dinosaurio: Chaoyangsaurus youngi, Clasificación: carnivore
Dinosaurio: Protarchaeopteryx robusta, Clasificación: herbivore
Dinosaurio: Caudipteryx zoui, Clasificación: herbivore
Dinosaurio: Gorgosaurus libratus, Clasificación: carnivore


Then I make predictions on my test set and save them in a variable y_pred, then I show 4 examples of the predictions that were made

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f'model accuracy: {accuracy}')
print('Clasification Inform:')
print(report)

model accuracy: 0.8803245436105477
Clasification Inform:
                     precision    recall  f1-score   support

          carnivore       0.87      0.91      0.89       212
carnivore, omnivore       0.93      0.88      0.90        16
          herbivore       0.91      0.87      0.89       243
herbivore, omnivore       0.60      0.60      0.60         5
           omnivore       0.50      0.67      0.57         9
          piscivore       0.75      0.75      0.75         8

           accuracy                           0.88       493
          macro avg       0.76      0.78      0.77       493
       weighted avg       0.88      0.88      0.88       493



Finally I carry out an evaluation of my model,
The first makes a measurement so that if my model took 4 dinosaurs and said that they were carnivores, what percentage of those 4 were actually carnivores.
In the recall, if my model takes 10 dinosaurs and classifies them as carnivores, but only 5 if they were carnivores, then it shows me a measure of what percentage these 5 numbers represent in all the carinivorous dinosaurs in my dataset.
The f1-score is an average of both measurements and my support is my total number of dinosaurs in each category.
Below I show a measure of my effectiveness, which is accuracy.
the "macro avg" is a simple average of the evaluation metrics (such as precision, recall and F1 score) calculated for each class separately and finally the "weighted avg" is a weighted average of the evaluation metrics, where the weight of each class is based on the proportion of examples in that class in the data set.

**Logic**

To explain the logic of the decision tree we will start with an example. Imagine you are making decisions like "Should I bring an umbrella today?" and you take into account different factors, such as weather, temperature and the probability of rain. A decision tree works in a similar way. It starts with a main question at the top of the tree, such as "Is it raining?", and then branches into more questions based on the answers as you go down the tree. Each answer leads you to a decision, such as "Take an umbrella" or "Don't bring an umbrella." In this way, a decision tree makes decisions based on specific characteristics and follows a path until reaching a conclusion.